# STAN48 Programming for Data Science

## Final project

As a conclusion to the course I decided to create simple implementation of a feed forward neural network using mainly Numpy and base Python. While there are already exquisite packages that offer these solutions (like Tensorflow and Pytorch), a step by step implementation of a neural network is still valuable for teaching basic programming concepts as well as basic neural network concepts.

As a general example I will use the [Kaggle Dogs vs. Cats](https://www.microsoft.com/en-us/download/details.aspx?id=54765) dataset to classify whether a given picture shows a cat or not. As the data set only includes two different options, we can assume the `not cat` option to be the same as `dog`. As mentioned, the intent is to have a general example to expose how the algorithm works, and the intricasies of the programming challenge, in other words, it is *not* my intention to implement a functioning neural network from scratch **and** a good model for classifying cats. 

## R or Python?

This notebook is originally made for Python. One of the requirements for this project was that whatever the choice of application to be developed, it should be done in both Python **and** R. Being that so, this notebook will focus on porting the Python code to R, and therefore, I will not bother in copy pasting the exact same explanations about the data or what a Neural Network is. For that purpose I urge the reader to check the Python notebook with the same name as this one. 

One practical element of this choice is that at times you will be met with cells starting with `%%writefile`, in those cases one should change the kernel of the notebook (since this command is native to IPyKernel, not R). All else should run smoothly through R.

### The structure

This project include several files. In this notebook you will find the application related functions, however, many of the base functions used for the calculations are left in a separate file that concentrates all the basic calculation functions. Without those dependencies this notebook will not function as it should. Some basic concepts regarding neural networks will be presented through the notebook, but the focus of this work is exposing the programming challenge behind neural networks.

### The data

The data set contains 25000 images of dogs and cats, but 59 of them were corrupted or in grayscale and, therefore, dropped. The classes are balanced and the angle, depth, light, and dimensions are not uniform. While originally a Kaggle competition data set, I opted to use the version made available by Microsoft because it did not pre divide the data giving me more freedom to split the sets as I please.

In [ ]:
%%writefile helper_functions.R
#one could potentially use the `sigmoid` library for the relu and sigmoid functions, but they are very simple to implement too.
relu<-function(Z){
    A<-max(0,Z)
    cache<-Z
    return(list(A,cache))
}
sigmoid<-function(Z){
    A<-1/(1+exp(Z))
    cache<-Z
    return(list(A,cache))
}
relu_backprop<-function(dA, cache){
    Z=cache
    dZ<-dA
    dZ[Z<=0]<-0
    return(dZ)
}
sigmoid_backprop<-function(dA,cache){
    Z<-cache
    s<-1/(1+exp(-Z))
    dZ<-dA*s*(1-s)
    return(dZ)
}

In [ ]:
%%writefile init_param.R
fill.matrix<-function(expr,nrow=1,ncol=1){
    matrix(eval(expr,envir=list(x=nrow*ncol)),nrow=nrow,ncol=ncol)
}
init_param<-function(layer_dim){
    parameters<-list()
    L<-length(layer_dim)
    for (l in 1:L){
        parameters[paste("W",l,sep="")]<-fill.matrix(rnorm(n=layer_dim[l]*layer_dim[l-1],mean=0,var=1)*0.01,layer_dim[l],layer_dim[l-1])
        parameters[paste("b",l,sep="")]<-matrix(0,layer_dim[l],1)
    }
    return(parameters)
}

In [ ]:
%%writefile for_prop.R
source("helper_functions.R")
for_prop<-funtion(A,W,b){
    Z<-(W%*%A)+b
    cache<-list(A,W,b)
    return(list(Z,cache))
}
for_activation<-function(A_prev,W,b,activ){
    if(activ=='sigmoid'){
        Z<-for_prop(A_prev,W,b)[[1]]
        linear_cache<-for_prop(A_prev,W,b)[[2]]
        A<-sigmoid(Z)[[1]]
        activ_cache<-sigmoid(Z)[[2]]
    } else if (activ=='relu'){
        Z<-for_prop(A_prev,W,b,activ)[[1]]
        linear_cache<-for_prop(A_prev,W,b)[[2]]
        A<-relu(Z)[[1]]
        activ_cache<-relu(Z)[[2]]
    }
    cache=list(linear_cache,activ_cache)
    return(list(A,cache))
}

In [ ]:
%%writefile deep_model.R
source("for_prop.R")
deep_model<-function(X,parameters){
    caches<-list()
    A<-X
    L<-length(parameters)%/%2
    for (l in 1:L){
        A_prev<-A
        A<-for_activation(A_prev,parameters[paste("W",l,sep="")],parameters[paste("b",l,sep="")],activ='relu')[[1]]
        cache<-for_activation(A_prev,parameters[paste("W",l,sep="")],parameters[paste("b",l,sep="")],activ='relu')[[2]]
        append(caches,cache)
    }
    AV<-for_activation(A_prev,parameters[paste("W",L,sep="")],parameters[paste("b",L,sep="")],activ='sigmoid')[[1]]
    cache<-for_activation(A_prev,parameters[paste("W",L,sep="")],parameters[paste("b",L,sep="")],activ='sigmoid')[[2]]
    append(caches,cache)
    return(list(AV,caches))
}

In [ ]:
%%writefile cost_computation.R
cost_computation<-function(AV,Y){
    m<-dim(Y)[2]
    cost=-(1/m)*sum(log(AV)%*%t(Y)+log((1-AV))%*%(1-t(Y)))
    cost<-unlist(cost)
    return(cost)
}

In [ ]:
%%writefile back_prop.R
source("helper_functions.R")
back_prop<-function(dZ,cache){
    A_prev<-cache[1]
    W<-cache[2]
    b<-cache[3]
    m<-dim(A_prev)[2]
    dW<-(1/m)*(dZ%*%t(A_prev))
    db<-(1/m)*sum(dZ)
    dA_prev<-t(W)%*%dZ
    return(list(dA_prev,dW,db))
}
back_activ<-function(dA,cache,activ){
    for_cache<-cache[[1]]
    activ_cache[[2]]
    if (activ=='relu'){
        dZ<-relu_backprop(dA,activ_cache)
        dA_prev<-back_prop(dZ,for_cache)[[1]]
        dW<-back_prop(dZ,for_cache)[[2]]
        db<-back_prop(dZ,for_cache)[[3]]
    } else if (activ=='sigmoid'){
        dZ<-sigmoid_backprop(dA,activ_cache)
        dA_prev<-back_prop(dZ,for_cache)[[1]]
        dW<-back_prop(dZ,for_cache)[[2]]
        db<-back_prop(dZ,for_cache)[[3]]
    }
    return(list(dA_prev,dW,db))
}
deep_model_back<-function(AV,Y,caches){
    grads<-list()
    L<-length(caches)
    m<-dim(AV)[2]
    Y<-matrix(Y, nrow=1, ncol=length(AV))
    dAL=-(Y/AV)-((1-Y)/(1-AV))
    present_cache<-caches[L-1]
    grads[paste("dA",L-1,sep="")]<-back_activ(dAL,present_cache,activ='sigmoid')[[1]]
    grads[paste("dW",L,sep="")]<-back_activ(dAL,present_cache,activ='sigmoid')[[2]]
    grads[paste('db',L,sep="")]<-back_activ(dAL,present_cache,activ='sigmoid')[[3]]
    for (l in (L-1):1){
        present_cache<-caches[l]
        dA_prev_temp<-back_activ(grads[paste("dA",l+1,sep="")],present_cache,activ='relu')[[1]]
        dW_temp<-back_activ(grads[paste('dA',l+1,sep="")],present_cache,activ='relu')[[2]]
        db_temp<-back_activ(grads[paste('dA',l+1,sep="")],present_cache,activ='relu')[[3]]
        grads[paste('dA',l,sep="")]<-dA_prev_temp
        grads[paste('dW',l,sep="")]<-dW_temp
        grads[paste('db',l,sep="")]<-db_temp
    }
    return(grads)
}

In [ ]:
%%writefile update.R
update<-function(params,grads,learning_rate){
    parameters<-params
    L<-length(parameters)%/%2
    for (l in range 1:L){
        parameters[paste("W",l+1,sep="")]<-parameters[paste("W",l+1,sep="")]-learning_rate*grads[paste('dW',l+1,sep="")]
        parameters[paste("b",l+1,sep="")]<-parameters[paste("b",l+1,sep="")]-learning_rate*grads[paste('db',l+1,sep="")]
    }
    return(parameters)
}

In [ ]:
%%writefile create_dataset.R
library(imager)
load <- function(im){
  skip_to_next<-FALSE
  tryCatch(load.image(im), error = function(e){skip_to_next<<-TRUE},
           if(skip_to_next){next})
}
clean_set<-function(loaded_images){
  for(i in 1:length(loaded_images)){
    error_check<-FALSE
    tryCatch(if(dim(loaded_images[[i]])[4]!=3){
      loaded_images <- loaded_images[-i]
    }, error=function(e){error_check<<-TRUE},
    if(error_check){loaded_images=loaded_images[-i]})
    tryCatch(if(dim(loaded_images[[i]])[4]==3){ 
      next},error=function(e){error_check<<-TRUE},
    if(error_check){loaded_images=loaded_images[-i]})
    tryCatch(if(loaded_images[[i]]==1){
      loaded_images <- loaded_images[-1]
    }, error=function(e){error_check<<-TRUE},
    if(error_check){next})
  }
  return(loaded_images)
}
resize_and_label<-function(loaded_images,label,width,height){
  for(i in 1:length(loaded_images)){
    loaded_images[[i]]<-resize(loaded_images[[i]],width,height,interpolation_type=1)
  }
  loaded_images<-as.matrix(loaded_images)
  loaded_images["labels"]<-rep(label, dim(loaded_images)[1])
  return(loaded_images)
}
create_dataset<-function(filenames_cat,filenames_dog, labels=c(1,0),width,height){
  loaded_cats<-sapply(filenames_cat,FUN=load,USE.NAMES=TRUE)
  names(loaded_cats)<-gsub("C:/Users/wtrindad/source/repos/NN_from_scratch/PetImages/","",names(loaded_cats))
  plot(loaded_cats$`Cat/1.jpg`)
  loaded_cats<-clean_set(loaded_cats)
  loaded_cats<-resize_and_label(loaded_cats,labels[1],width,height)
  plot(loaded_cats[[2]])
  loaded_dogs<-sapply(filenames_dog,FUN=load,USE.NAMES=TRUE)
  names(loaded_dogs)<-gsub("C:/Users/wtrindad/source/repos/NN_from_scratch/PetImages/","",names(loaded_dogs))
  plot(loaded_dogs$`Dog/1.jpg`)
  loaded_dogs<-clean_set(loaded_dogs)
  loaded_dogs<-resize_and_label(loaded_dogs,labels[2],width,height)
  plot(loaded_dogs[[2]])
  dataset<-rbind(loaded_cats,loaded_dogs)
  return(dataset)
}

In [ ]:
%%writefile dense_nn.R
source("update.R")
source("back_prop.R")
source("deep_model.R")
source("init_param.R")
source("cost_computation.R")

dense_nn<-function(X,Y,layers_dims,learning_rate=0.0075,num_iterations=5000,print_cost=False){
    costs<-list()
    parameters<-init_param(layers_dims)
    for(i in 1:num_interations){
        AV<-deep_model(X,parameters)[[1]]
        caches<-deep_model(X,parameters)[[2]]
        cost<-cost_computation(AV,Y)
        grads<-deep_model_back(AV,Y,caches)
        parameters<-update(parameters,grads,learning_rate)
        if(print_cost && i%%100==0 or i==num_interations-1){
            print("Cost after iteration ",i,": ",cost)
        }
        if(i%%100==0 or i==num_iterations){
            append(costs,cost)
        }
    }
    return(list(parameters,costs))
}

In [ ]:
%%writefile predict.py
source("deep_model.R")
predict<-function(X,y,parameters){
    m<-dim(X)[2]
    n<-length(parameters)%/%2
    p<-matrix(0,1,m)
    dm<-deep_model(X,parameters)
    probs<-dm[[1]]
    caches<-dm[[2]]
    for (i in 1:dim())
}
def predict(X,y,parameters):
    """
    Takes X (the data), y (the labels), and parameters (from the dense_nn() function).
    Returns predictions p for the chosen data set X.
    """
    m=X.shape[1]
    n=len(parameters)//2
    p=np.zeros((1,m))
    probs,caches=deep_model(X, parameters)
    for i in range(0,probs.shape[1]):
        if probs[0,i]>0.5:
            p[0,i]=1
        else:
            p[0,i]=0
    print("Accuracy: "  + str(np.sum((p == y)/m)))   
    return p

In [ ]:
source("create_dataset.R")
filenames_cat <- list.files("C:/Users/wtrindad/source/repos/NN_from_scratch/PetImages/Cat/", pattern="*.jpg", full.names=T)
filenames_dog <- list.files("C:/Users/wtrindad/source/repos/NN_from_scratch/PetImages/Dog/", pattern="*.jpg", full.names=T)
img_data<-create_dataset(filenames_cat,filenames_dog,labels=c(1,0),width=32,height=32) # Do notice this cell will take a good 
                                                                                       # 10min or more to run, more if your computer 
                                                                                       # is below average. As it progresses it will 
                                                                                       # plot different images, you can use these as 
                                                                                       # reference of what stage of the calculation
                                                                                       # the function is at. Also to see cute animals
                                                                                        

In [ ]:
samples<-sample(c(TRUE,FALSE),nrow(img_data),replace=TRUE,prob=c(0.85,0.15))
x_train<-img_data[sample,1]
y_train<-img_data[sample,2]
x_test<-img_data[!sample,1]
y_test<-img_data[!sample,2]

In [ ]:
layer_dims<-c(length(x_train),20,7,5,1) #?????

In [ ]:
source("dense_nn.R")
short_run<-dense_nn(x_train,y_train,layer_dims,num_iterations=1,print_cost=True)
parameters<-short_run[[1]]
costs<-short_run[[2]]

In [ ]:
long_run<-dense_nn(x_train,y_train,layer_dims,num_iterations=500,print_cost=True)
parameters<-long_run[[1]]
costs<-long_run[[2]]

In [ ]:
from predict import predict
predict_train=predict(x_train_flat,y_train,parameters)

In [ ]:
predict_test=predict(x_test_flat,y_test,parameters)